# Project

In [1]:
import os
import pytimeloop.timeloopfe.v4 as tl

In [2]:
def run_timeloop_model(architecture, mapping, problem, config: dict = None):
    if os.path.exists("./output_dir"):
        os.system("rm -r ./output_dir")

    if config:
        spec = tl.Specification.from_yaml_files(architecture, mapping, problem, jinja_parse_data=config)
    else:
        spec = tl.Specification.from_yaml_files(architecture, mapping, problem)
    
    return tl.call_model(spec, output_dir="./output_dir")

In [3]:
def run_timeloop_mapper(architecture, problem, constraints, mapper, config: dict = None):
    if os.path.exists("./output_dir"):
        os.system("rm -r ./output_dir")

    if config:
        spec = tl.Specification.from_yaml_files(architecture, problem, constraints, mapper, jinja_parse_data=config)
    else:
        spec = tl.Specification.from_yaml_files(architecture, problem, constraints, mapper)

    return tl.call_mapper(spec, output_dir="./output_dir")

In [4]:
problems = [
    'layer_shapes/alexnet/0.yaml',
    'layer_shapes/alexnet/1.yaml',
    'layer_shapes/alexnet/2.yaml',
    'layer_shapes/alexnet/3.yaml',
    'layer_shapes/alexnet/4.yaml',
    'layer_shapes/alexnet/5.yaml',
    'layer_shapes/alexnet/6.yaml',
    'layer_shapes/alexnet/7.yaml'
]

num_gpus = 16

### Data-Parallel

In [5]:
for problem in problems:
    result = run_timeloop_mapper(
        architecture='designs/system/arch.yaml',
        problem=problem,
        constraints='designs/system/constraints_dp.yaml',
        mapper='designs/_include/mapper.yaml',
        config={'gpu_meshX': num_gpus}
    )
    
    stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()

    # print(stats)
    print(result.mapping)

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    
    print(problem, energy, cycles)

SystemMemory [ Weights:23232 (23232) Inputs:811200 (811200) Outputs:12390400 (12390400) ] 
-----------------------------------------------------------------------------------------
| for N in [0:4)

inter_GPU_spatial [ ] 
---------------------
|   for N in [0:16) (Spatial-X)

LocalMemory [ Weights:23232 (23232) Inputs:12675 (12675) Outputs:193600 (193600) ] 
----------------------------------------------------------------------------------
|     for M in [0:64)

LocalBuffer [ Weights:363 (363) Inputs:12675 (12675) Outputs:3025 (3025) ] 
inter_PE_spatial [ ] 
--------------------
|       for C in [0:3) (Spatial-X)

RegisterFile [ Weights:121 (121) ] 
----------------------------------
|         for Q in [0:55)
|           for P in [0:55)
|             for S in [0:11)
|               for R in [0:11)
|                 << Compute >>

layer_shapes/alexnet/0.yaml 42574.06 93702400
SystemMemory [ Weights:307200 (307200) Inputs:3936256 (3936256) Outputs:8957952 (8957952) ] 
-------------------

### Tensor-Parallel

In [6]:
for problem in problems:
    result = run_timeloop_mapper(
        architecture='designs/system/arch.yaml',
        problem=problem,
        constraints='designs/system/constraints_tp.yaml',
        mapper='designs/_include/mapper.yaml',
        config={'gpu_meshX': num_gpus}
    )
    
    stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()

    # print(stats)
    print(result.mapping)

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    
    print(problem, energy, cycles)

SystemMemory [ Weights:23232 (23232) Inputs:811200 (811200) Outputs:12390400 (12390400) ] 
inter_GPU_spatial [ ] 
---------------------
| for M in [0:16) (Spatial-X)

LocalMemory [ Weights:1452 (1452) Inputs:811200 (811200) Outputs:774400 (774400) ] 
----------------------------------------------------------------------------------
|   for N in [0:64)
|     for M in [0:4)

LocalBuffer [ Weights:363 (363) Inputs:12675 (12675) Outputs:3025 (3025) ] 
inter_PE_spatial [ ] 
--------------------
|       for C in [0:3) (Spatial-X)

RegisterFile [ Weights:121 (121) ] 
----------------------------------
|         for Q in [0:55)
|           for P in [0:55)
|             for S in [0:11)
|               for R in [0:11)
|                 << Compute >>

layer_shapes/alexnet/0.yaml 45686.34 93702400
SystemMemory [ Weights:307200 (307200) Inputs:3936256 (3936256) Outputs:8957952 (8957952) ] 
inter_GPU_spatial [ ] 
---------------------
| for M in [0:16) (Spatial-X)

LocalMemory [ Weights:19200 (19200